# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import time


# Import API key
from api_keys_kuhl import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Pressure,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Gunnison,38.7000,-107.0673,-23.53,1034,78,7,2.52,US,1733016385
1,1,Afaahiti,-17.7500,-149.2833,26.59,1011,77,60,3.99,PF,1733016385
2,2,Ban Karon,7.8477,98.2985,28.77,1010,84,99,2.75,TH,1733016385
3,3,Iqaluit,63.7506,-68.5145,-6.15,1020,57,75,6.17,CA,1733016386
4,4,Port-aux-Français,-49.3500,70.2167,5.84,984,95,100,12.89,TF,1733016386


In [3]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     577 non-null    int64  
 1   City        577 non-null    object 
 2   Lat         577 non-null    float64
 3   Lng         577 non-null    float64
 4   Max Temp    577 non-null    float64
 5   Pressure    577 non-null    int64  
 6   Humidity    577 non-null    int64  
 7   Cloudiness  577 non-null    int64  
 8   Wind Speed  577 non-null    float64
 9   Country     571 non-null    object 
 10  Date        577 non-null    int64  
dtypes: float64(4), int64(5), object(2)
memory usage: 49.7+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600,
    title = "City Humidity Map",
    hoover_cols=["City", "Country", "Humidity"]
    
)

# Display the map plot
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 70
wind_speed = 5
humidity = 40

# Drop any rows with null values
df2 = city_data_df.dropna()

mask = ((df2["Max Temp"] <= max_temp) & (df2["Wind Speed"] <= wind_speed) & (df2["Humidity"] <= humidity))
df2 = df2.loc[mask].reset_index(drop=True)
# Display sample data
df2

,City_ID,City,Lat,Lng,Max Temp,Pressure,Humidity,Cloudiness,Wind Speed,Country,Date
0,27,Hasaki,35.7333,140.8333,14.61,1015,40,0,3.05,JP,1733016389
1,80,Sonoita,31.8500,-112.8333,19.74,1016,24,0,2.68,MX,1733016397
2,83,Safi,32.1667,-8.8333,18.78,1023,37,100,1.27,MA,1733016397
3,86,Salé,34.0389,-6.8166,26.38,1021,24,100,3.60,MA,1733016397
4,107,Kayes,14.0000,-11.0000,22.89,1014,16,0,2.46,ML,1733016399
5,124,Crane,31.3974,-102.3501,13.18,1020,35,100,2.55,US,1733016401
6,131,Nouadhibou,20.9310,-17.0347,22.99,1018,38,7,3.60,MR,1733016401
7,195,Chandler,33.3062,-111.8412,22.04,1016,29,0,3.09,US,1733016190
8,200,Evergreen,39.6333,-105.3172,3.09,1019,37,100,1.34,US,1733016414
9,201,Reggane,26.7158,0.1714,17.36,1021,25,78,4.83,DZ,1733016414


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df2[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy() 
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = np.nan

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Hasaki,JP,35.7333,140.8333,40,NaN
1,Sonoita,MX,31.8500,-112.8333,24,NaN
2,Safi,MA,32.1667,-8.8333,37,NaN
3,Salé,MA,34.0389,-6.8166,24,NaN
4,Kayes,ML,14.0000,-11.0000,16,NaN
5,Crane,US,31.3974,-102.3501,35,NaN
6,Nouadhibou,MR,20.9310,-17.0347,38,NaN
7,Chandler,US,33.3062,-111.8412,29,NaN
8,Evergreen,US,39.6333,-105.3172,37,NaN
9,Reggane,DZ,26.7158,0.1714,25,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 #10km
params = {
    "categories":"accomodation.hotel",
    "apiKey": geoapify_key,
    "limit":56
}
    

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Longitude = row["Lng"]
    Latitude =row["Lat"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    #Time lag 1 Second
    time.sleep(1)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"], 
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


C:\Users\kuhlj\AppData\Local\Temp\ipykernel_24472\2151480723.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'No hotel found' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  hotel_df.loc[index, "Hotel Name"] = "No hotel found"


Hasaki - nearest hotel: No hotel found
Sonoita - nearest hotel: No hotel found
Safi - nearest hotel: No hotel found
Salé - nearest hotel: No hotel found
Kayes - nearest hotel: No hotel found
Crane - nearest hotel: No hotel found
Nouadhibou - nearest hotel: No hotel found
Chandler - nearest hotel: No hotel found
Evergreen - nearest hotel: No hotel found
Reggane - nearest hotel: No hotel found
Kongoussi - nearest hotel: No hotel found
Mora - nearest hotel: No hotel found
Hūn - nearest hotel: No hotel found
Muzaffargarh - nearest hotel: No hotel found
Tabas - nearest hotel: No hotel found
Tindouf - nearest hotel: No hotel found
Bobo-Dioulasso - nearest hotel: No hotel found
Hārij - nearest hotel: No hotel found
Laojunmiao - nearest hotel: No hotel found
Bharatpur - nearest hotel: No hotel found
Mhamid - nearest hotel: No hotel found
Adrar - nearest hotel: No hotel found
Zouérat - nearest hotel: No hotel found
San Julián - nearest hotel: No hotel found
Koné - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,Hasaki,JP,35.7333,140.8333,40,No hotel found
1,Sonoita,MX,31.8500,-112.8333,24,No hotel found
2,Safi,MA,32.1667,-8.8333,37,No hotel found
3,Salé,MA,34.0389,-6.8166,24,No hotel found
4,Kayes,ML,14.0000,-11.0000,16,No hotel found
5,Crane,US,31.3974,-102.3501,35,No hotel found
6,Nouadhibou,MR,20.9310,-17.0347,38,No hotel found
7,Chandler,US,33.3062,-111.8412,29,No hotel found
8,Evergreen,US,39.6333,-105.3172,37,No hotel found
9,Reggane,DZ,26.7158,0.1714,25,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# No hotels found in unable to render accurate map

In [9]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Hotels",
    color = "City",
    frame_width = 800,
    frame_height = 600,
    title = "Hotel Map",
    hoover_cols=["City", "Country", "Hotel Name"]
    
)

# Display the map plot
map_plot2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)